In [1]:
from data_loader import *
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as func
import torch.nn.init as torch_init
import torch.optim as optim
import shutil
import pickle

class LSTM(nn.Module):
    """ A basic LSTM model. 
    """
    
    def __init__(self, in_dim, out_dim, hid_dim, batch_size, no_layers =1):
        super(LSTM, self).__init__()
        #specify the input dimensions
        self.in_dim = in_dim
        #specify the output dimensions
        self.out_dim = out_dim
        #specify hidden layer dimensions
        self.hid_dim = hid_dim
        #specify the number of layers
        self.no_layers = no_layers  
        #self.batch_size=batch_size
        
        #initialise the LSTM
        self.model = nn.LSTM(self.in_dim, self.hid_dim, self.no_layers)
        self.outputs = nn.Linear(self.hid_dim, out_dim)

    def forward(self, batch,hidden=None):
        """Pass the batch of images through each layer of the network, applying 
        """
        
        lstm_out, hidden = self.model(batch, hidden)
        y_pred = self.outputs(lstm_out.view(lstm_out.shape[0]*lstm_out.shape[1],lstm_out.shape[-1]))
        #pdb.set_trace()
        #The input is expected to contain raw, unnormalized scores for each class according to documentation
        #tag_scores = func.softmax(y_pred,dim=2)
        #return tag_scores,hidden
        return y_pred,hidden

    
def validate(val_loader, model, criterion):
    losses = AverageMeter()
    acc = AverageMeter()

    # switch to evaluate mode
    model.eval()  
    hidden=None
    for minibatch_count, (images, labels) in enumerate(val_loader, 0):
        images=images.permute(1,0,2)
        # Put the minibatch data in CUDA Tensors and run on the GPU if supported
        images, labels = images.to(computing_device), labels.to(computing_device)
        images=images.float()
       
        outputs,hidden = model(images,hidden)
        hidden[0].detach_() 
        hidden[1].detach_()
        
        labels=labels.view(-1)
        loss = criterion(outputs, labels)
        
        softmaxOutput=func.softmax(outputs,dim=1)
        modelDecision=torch.argmax(softmaxOutput,dim=1)
        accuracy = calculate_acc(modelDecision,labels)
        losses.update(loss.item(), labels.shape[0])
        acc.update(accuracy, labels.shape[0])

    return acc,losses
class AverageMeter(object):

    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

def calculate_acc(modelDecision,labels):
    acc=torch.sum(labels==modelDecision).to(dtype=torch.float)/labels.shape[0]
    return acc

def save_checkpoint(state, is_best, filename='checkpoint.pth.tar'):
    torch.save(state, filename)
    if is_best:
        shutil.copyfile(filename, 'model_best.pth.tar')
        
        
# Setup: initialize the hyperparameters/variables
num_epochs = 30           # Number of full passes through the dataset
batch_size = 1          # Number of samples in each minibatch
learning_rate = 0.001  
#use_cuda=0
use_cuda=torch.cuda.is_available()

# Setup GPU optimization if CUDA is supported
if use_cuda:
    computing_device = torch.device("cuda")
    extras = {"num_workers": 1, "pin_memory": True}
    print("CUDA is supported")
else: # Otherwise, train on the CPU
    computing_device = torch.device("cpu")
    extras = False
    print("CUDA NOT supported")

# Setup the training, validation, and testing dataloaders
train_loader, val_loader, test_loader = create_split_loaders(batch_size,shuffle=False, show_sample=False,extras=extras)

# Instantiate a BasicCNN to run on the GPU or CPU based on CUDA support
model = LSTM(in_dim=94, out_dim=94,hid_dim=100,batch_size=16,no_layers=1)
model = model.to(computing_device)
print("Model on CUDA?", next(model.parameters()).is_cuda)

#TODO: Define the loss criterion and instantiate the gradient descent optimizer
criterion = torch.nn.CrossEntropyLoss() #TODO - loss criteria are defined in the torch.nn package

#TODO: Instantiate the gradient descent optimizer - use Adam optimizer with default parameters
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)


# Track the loss across training
total_loss = []
avg_minibatch_loss = []
best_loss = 100
acc_train_list=[]
acc_val_list=[]

loss_train_list=[]
loss_val_list=[]

hidden=None

# Begin training procedure
for epoch in range(num_epochs):
    model.train(True)
    N = 50
    N_minibatch_loss = 0.0    
    train_acc = AverageMeter()
    train_precision = AverageMeter()
    train_recall = AverageMeter()
    train_BCR = AverageMeter()
    # Get the next minibatch of images, labels for training
    for minibatch_count, (images, labels) in enumerate(train_loader, 0):

        images=images.permute(1,0,2)

        
        # Put the minibatch data in CUDA Tensors and run on the GPU if supported
        images, labels = images.to(computing_device), labels.to(computing_device)
        images=images.float()
        # Zero out the stored gradient (buffer) from the previous iteration
        optimizer.zero_grad()
        

        # Perform the forward pass through the network and compute the loss
        outputs,hidden = model(images,hidden)
        hidden[0].detach_() 
        hidden[1].detach_()
#         outputs.shape => batchSize * sequenceSize *dictionarySize
#         labels.shape => batchSize * sequenceSize
        labels=labels.view(-1)
        loss = criterion(outputs,labels)
        
        # Automagically compute the gradients and backpropagate the loss through the network
        #loss.backward(retain_graph=True)
        loss.backward()
        # Update the weights
        optimizer.step()

        #Calculate accuracy
        #pdb.set_trace()
        softmaxOutput=func.softmax(outputs,dim=1)
        modelDecision=torch.argmax(softmaxOutput,dim=1)
        accuracy = calculate_acc(modelDecision,labels)
        train_acc.update(accuracy, labels.shape[0])
        

        # Add this iteration's loss to the total_loss
        total_loss.append(loss.item())
        N_minibatch_loss += float(loss)
        
        #TODO: Implement cross-validation
        del outputs
        del loss
        
        if minibatch_count % N == 0:    
            
            # Print the loss averaged over the last N mini-batches    
            N_minibatch_loss /= N
            print('Epoch %d, average minibatch %d loss: %.3f, average accuracy: %.3f' %
                (epoch + 1, minibatch_count, N_minibatch_loss,train_acc.avg.mean()))
            
            # Add the averaged loss over N minibatches and reset the counter
            avg_minibatch_loss.append(N_minibatch_loss)
            N_minibatch_loss = 0.0

            
    print("Finished", epoch + 1, "epochs of training")
    #TODO: Implement cross-validation
    with torch.no_grad():
        acc_val,loss_val = validate(val_loader, model, criterion)
    
    acc_train_list.append(train_acc.avg.mean())
    acc_val_list.append((acc_val.avg).mean())
    
    loss_val_list.append(loss_val.avg)
    print('val_loss: %.3f, average accuracy of all classes: %.3f'%(loss_val.avg,(acc_val.avg).mean()))
    
    # remember best loss and save checkpoint
    is_best = loss_val.avg <= best_loss
    best_loss = min(loss_val.avg, best_loss)
    save_checkpoint({
        'epoch': epoch + 1,
        'state_dict': model.state_dict(),
        'best_loss': best_loss,
        'optimizer': optimizer.state_dict(),
    }, is_best)
    
print("Training complete after", epoch, "epochs")

results = { "acc_train_list": acc_train_list, "acc_val_list": acc_val_list, "loss_train_list":loss_train_list,"loss_val_list":loss_val_list}
pickle.dump( results, open( "own_results_base.p", "wb" ) )

CUDA is supported
[27 81 82 ... 67 76 66]
[81 82 63 ... 76 66 29]
[27 81 82 ... 67 76 66]
[81 82 63 ... 76 66 29]
[27 81 82 ... 67 76 66]
[81 82 63 ... 76 66 29]
Model on CUDA? True
Epoch 1, average minibatch 0 loss: 0.091, average accuracy: 0.010
Epoch 1, average minibatch 50 loss: 3.988, average accuracy: 0.140
Epoch 1, average minibatch 100 loss: 3.143, average accuracy: 0.159
Epoch 1, average minibatch 150 loss: 3.653, average accuracy: 0.156
Epoch 1, average minibatch 200 loss: 3.544, average accuracy: 0.157
Epoch 1, average minibatch 250 loss: 3.354, average accuracy: 0.166
Epoch 1, average minibatch 300 loss: 3.410, average accuracy: 0.166
Epoch 1, average minibatch 350 loss: 3.638, average accuracy: 0.167
Epoch 1, average minibatch 400 loss: 3.374, average accuracy: 0.165
Epoch 1, average minibatch 450 loss: 3.533, average accuracy: 0.160
Epoch 1, average minibatch 500 loss: 3.365, average accuracy: 0.161
Epoch 1, average minibatch 550 loss: 3.281, average accuracy: 0.159
Epoch

Epoch 2, average minibatch 2000 loss: 1.848, average accuracy: 0.491
Epoch 2, average minibatch 2050 loss: 1.401, average accuracy: 0.494
Epoch 2, average minibatch 2100 loss: 1.250, average accuracy: 0.498
Epoch 2, average minibatch 2150 loss: 1.768, average accuracy: 0.498
Epoch 2, average minibatch 2200 loss: 1.840, average accuracy: 0.498
Epoch 2, average minibatch 2250 loss: 1.689, average accuracy: 0.498
Epoch 2, average minibatch 2300 loss: 1.826, average accuracy: 0.498
Epoch 2, average minibatch 2350 loss: 1.605, average accuracy: 0.499
Epoch 2, average minibatch 2400 loss: 1.573, average accuracy: 0.500
Epoch 2, average minibatch 2450 loss: 2.002, average accuracy: 0.499
Epoch 2, average minibatch 2500 loss: 1.852, average accuracy: 0.498
Epoch 2, average minibatch 2550 loss: 1.767, average accuracy: 0.497
Epoch 2, average minibatch 2600 loss: 1.848, average accuracy: 0.496
Epoch 2, average minibatch 2650 loss: 1.713, average accuracy: 0.496
Epoch 2, average minibatch 2700 lo

Epoch 4, average minibatch 250 loss: 1.502, average accuracy: 0.512
Epoch 4, average minibatch 300 loss: 1.775, average accuracy: 0.515
Epoch 4, average minibatch 350 loss: 1.635, average accuracy: 0.522
Epoch 4, average minibatch 400 loss: 1.662, average accuracy: 0.522
Epoch 4, average minibatch 450 loss: 1.238, average accuracy: 0.536
Epoch 4, average minibatch 500 loss: 1.067, average accuracy: 0.552
Epoch 4, average minibatch 550 loss: 1.477, average accuracy: 0.552
Epoch 4, average minibatch 600 loss: 1.875, average accuracy: 0.546
Epoch 4, average minibatch 650 loss: 1.471, average accuracy: 0.548
Epoch 4, average minibatch 700 loss: 1.344, average accuracy: 0.554
Epoch 4, average minibatch 750 loss: 1.249, average accuracy: 0.560
Epoch 4, average minibatch 800 loss: 1.448, average accuracy: 0.561
Epoch 4, average minibatch 850 loss: 1.166, average accuracy: 0.567
Epoch 4, average minibatch 900 loss: 1.431, average accuracy: 0.569
Epoch 4, average minibatch 950 loss: 1.275, aver

Epoch 5, average minibatch 2400 loss: 1.242, average accuracy: 0.586
Epoch 5, average minibatch 2450 loss: 1.629, average accuracy: 0.585
Epoch 5, average minibatch 2500 loss: 1.534, average accuracy: 0.584
Epoch 5, average minibatch 2550 loss: 1.417, average accuracy: 0.583
Epoch 5, average minibatch 2600 loss: 1.559, average accuracy: 0.582
Epoch 5, average minibatch 2650 loss: 1.418, average accuracy: 0.581
Epoch 5, average minibatch 2700 loss: 1.678, average accuracy: 0.580
Epoch 5, average minibatch 2750 loss: 1.262, average accuracy: 0.580
Epoch 5, average minibatch 2800 loss: 1.421, average accuracy: 0.580
Epoch 5, average minibatch 2850 loss: 1.490, average accuracy: 0.579
Epoch 5, average minibatch 2900 loss: 1.480, average accuracy: 0.578
Epoch 5, average minibatch 2950 loss: 1.377, average accuracy: 0.578
Epoch 5, average minibatch 3000 loss: 1.575, average accuracy: 0.577
Epoch 5, average minibatch 3050 loss: 1.405, average accuracy: 0.577
Epoch 5, average minibatch 3100 lo

Epoch 7, average minibatch 650 loss: 1.202, average accuracy: 0.601
Epoch 7, average minibatch 700 loss: 1.167, average accuracy: 0.606
Epoch 7, average minibatch 750 loss: 1.108, average accuracy: 0.611
Epoch 7, average minibatch 800 loss: 1.272, average accuracy: 0.611
Epoch 7, average minibatch 850 loss: 1.010, average accuracy: 0.616
Epoch 7, average minibatch 900 loss: 1.230, average accuracy: 0.618
Epoch 7, average minibatch 950 loss: 1.154, average accuracy: 0.621
Epoch 7, average minibatch 1000 loss: 1.338, average accuracy: 0.620
Epoch 7, average minibatch 1050 loss: 1.258, average accuracy: 0.620
Epoch 7, average minibatch 1100 loss: 1.382, average accuracy: 0.620
Epoch 7, average minibatch 1150 loss: 1.496, average accuracy: 0.618
Epoch 7, average minibatch 1200 loss: 1.335, average accuracy: 0.617
Epoch 7, average minibatch 1250 loss: 1.196, average accuracy: 0.619
Epoch 7, average minibatch 1300 loss: 1.285, average accuracy: 0.619
Epoch 7, average minibatch 1350 loss: 1.4

...

Epoch 24, average minibatch 150 loss: 1.148, average accuracy: 0.660
Epoch 24, average minibatch 200 loss: 1.078, average accuracy: 0.660
Epoch 24, average minibatch 250 loss: 1.119, average accuracy: 0.661
Epoch 24, average minibatch 300 loss: 1.214, average accuracy: 0.660
Epoch 24, average minibatch 350 loss: 1.170, average accuracy: 0.661
Epoch 24, average minibatch 400 loss: 1.138, average accuracy: 0.661
Epoch 24, average minibatch 450 loss: 0.860, average accuracy: 0.670
Epoch 24, average minibatch 500 loss: 0.776, average accuracy: 0.680
Epoch 24, average minibatch 550 loss: 1.033, average accuracy: 0.681
Epoch 24, average minibatch 600 loss: 1.354, average accuracy: 0.674
Epoch 24, average minibatch 650 loss: 0.924, average accuracy: 0.678
Epoch 24, average minibatch 700 loss: 0.892, average accuracy: 0.682
Epoch 24, average minibatch 750 loss: 0.913, average accuracy: 0.685
Epoch 24, average minibatch 800 loss: 1.037, average accuracy: 0.685
Epoch 24, average minibatch 850 lo

Epoch 25, average minibatch 2200 loss: 1.212, average accuracy: 0.686
Epoch 25, average minibatch 2250 loss: 1.072, average accuracy: 0.685
Epoch 25, average minibatch 2300 loss: 1.136, average accuracy: 0.684
Epoch 25, average minibatch 2350 loss: 0.963, average accuracy: 0.684
Epoch 25, average minibatch 2400 loss: 0.963, average accuracy: 0.684
Epoch 25, average minibatch 2450 loss: 1.236, average accuracy: 0.683
Epoch 25, average minibatch 2500 loss: 1.150, average accuracy: 0.682
Epoch 25, average minibatch 2550 loss: 1.039, average accuracy: 0.682
Epoch 25, average minibatch 2600 loss: 1.184, average accuracy: 0.681
Epoch 25, average minibatch 2650 loss: 1.039, average accuracy: 0.681
Epoch 25, average minibatch 2700 loss: 1.244, average accuracy: 0.680
Epoch 25, average minibatch 2750 loss: 0.944, average accuracy: 0.680
Epoch 25, average minibatch 2800 loss: 1.077, average accuracy: 0.680
Epoch 25, average minibatch 2850 loss: 1.093, average accuracy: 0.679
Epoch 25, average mi

Epoch 27, average minibatch 400 loss: 1.122, average accuracy: 0.668
Epoch 27, average minibatch 450 loss: 0.848, average accuracy: 0.676
Epoch 27, average minibatch 500 loss: 0.767, average accuracy: 0.686
Epoch 27, average minibatch 550 loss: 1.021, average accuracy: 0.686
Epoch 27, average minibatch 600 loss: 1.327, average accuracy: 0.679
Epoch 27, average minibatch 650 loss: 0.911, average accuracy: 0.683
Epoch 27, average minibatch 700 loss: 0.883, average accuracy: 0.687
Epoch 27, average minibatch 750 loss: 0.903, average accuracy: 0.689
Epoch 27, average minibatch 800 loss: 1.026, average accuracy: 0.689
Epoch 27, average minibatch 850 loss: 0.786, average accuracy: 0.693
Epoch 27, average minibatch 900 loss: 0.969, average accuracy: 0.694
Epoch 27, average minibatch 950 loss: 0.923, average accuracy: 0.696
Epoch 27, average minibatch 1000 loss: 1.071, average accuracy: 0.695
Epoch 27, average minibatch 1050 loss: 1.046, average accuracy: 0.694
Epoch 27, average minibatch 1100

Epoch 28, average minibatch 2450 loss: 1.196, average accuracy: 0.685
Epoch 28, average minibatch 2500 loss: 1.133, average accuracy: 0.684
Epoch 28, average minibatch 2550 loss: 1.022, average accuracy: 0.684
Epoch 28, average minibatch 2600 loss: 1.162, average accuracy: 0.683
Epoch 28, average minibatch 2650 loss: 1.011, average accuracy: 0.683
Epoch 28, average minibatch 2700 loss: 1.226, average accuracy: 0.682
Epoch 28, average minibatch 2750 loss: 0.915, average accuracy: 0.682
Epoch 28, average minibatch 2800 loss: 1.063, average accuracy: 0.682
Epoch 28, average minibatch 2850 loss: 1.068, average accuracy: 0.681
Epoch 28, average minibatch 2900 loss: 1.144, average accuracy: 0.681
Epoch 28, average minibatch 2950 loss: 1.038, average accuracy: 0.681
Epoch 28, average minibatch 3000 loss: 1.139, average accuracy: 0.680
Epoch 28, average minibatch 3050 loss: 1.015, average accuracy: 0.680
Epoch 28, average minibatch 3100 loss: 1.076, average accuracy: 0.680
Epoch 28, average mi

Epoch 30, average minibatch 650 loss: 0.895, average accuracy: 0.685
Epoch 30, average minibatch 700 loss: 0.882, average accuracy: 0.689
Epoch 30, average minibatch 750 loss: 0.896, average accuracy: 0.692
Epoch 30, average minibatch 800 loss: 1.019, average accuracy: 0.691
Epoch 30, average minibatch 850 loss: 0.778, average accuracy: 0.695
Epoch 30, average minibatch 900 loss: 0.955, average accuracy: 0.696
Epoch 30, average minibatch 950 loss: 0.908, average accuracy: 0.699
Epoch 30, average minibatch 1000 loss: 1.057, average accuracy: 0.698
Epoch 30, average minibatch 1050 loss: 1.037, average accuracy: 0.697
Epoch 30, average minibatch 1100 loss: 1.094, average accuracy: 0.696
Epoch 30, average minibatch 1150 loss: 1.117, average accuracy: 0.695
Epoch 30, average minibatch 1200 loss: 1.013, average accuracy: 0.695
Epoch 30, average minibatch 1250 loss: 0.961, average accuracy: 0.696
Epoch 30, average minibatch 1300 loss: 0.972, average accuracy: 0.696
Epoch 30, average minibatch

In [2]:
dictionary=pickle.load(open("dictionary_new.pkl","rb"))
from bidict import bidict 
reverse_dict = bidict(dictionary)
reverse_dict.inverse

def sample(net, size, prime, dictionary):
        
    if(use_cuda):
        computing_device = torch.device("cuda")
    else:
        computing_device = torch.device("cpu")
    net = net.to(computing_device)
    net.eval() 
    
    h = None
    chars=list(prime)
    lis=[]
    for ch in chars:
        a=dictionary[ch]
        lis.append(a)
    
    lis=np.array(lis)
    m=torch.nn.Softmax()
    a=torch.tensor(np.squeeze(np.eye(94)[lis.reshape(-1)]).reshape([len(prime), 1, 94])).to(computing_device).float()
    with torch.no_grad():
        char, h = net(a, None )        

        for i in range(char.size()[0]):
            o=m(char[i])
            p_p=o.cpu()
            p_p = np.array(p_p)
            p_p /= p_p.sum()          

         
        for j in range(size):
            last_output=char[-1].view((1,1,94))
           
            char,h=net(last_output,h)
            for i in range(char.size()[0]):
                o=m(char[i])
                p_p=o.cpu()
                p_p = np.array(p_p)
                p_p /= p_p.sum()   
                
                chars.append(reverse_dict.inverse[(np.random.choice(94, 1,p=p_p))[0]])
    return chars
        
result=(sample(model, 100000, '<start>', dictionary))

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:29: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:40: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


In [12]:
with open('generated.txt', 'w') as f:
    for item in result:
        f.write("%s" % item)